In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-07-08 19:57:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.30MB/s    in 0.2s    

2021-07-08 19:57:06 (5.30 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark import SparkConf                                                                                                                 
from pyspark.context import SparkContext                                                                                                      
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.appName("yelpDataCleaning").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()



In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
#url ="https://data-bootcamp-priya.s3.us-east-2.amazonaws.com/yelp_academic_dataset_business.json"
spark.sparkContext.addFile(url)
dataset_business_df = spark.read.json(SparkFiles.get("yelp_academic_dataset_business.json"))

# Show DataFrame
dataset_business_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|        921 Pearl St|[,, 'beer_and_win...|6iYb2HFDywm3zjuRg...|Gastropubs, Food,...|       Boulder|[11:0-23:0, 11:0-...|      1|   40.0175444|   -105.2833481| Oskar Blues Taproom|      80302|          86|  4.0|   CO|
| 7000 NE Airport Way|[,, u'beer_and_wi...|tCbdrRPZA0oiIYSmH...|Salad, Soup, Sand...|      Portland|[5:0-18:0, 5:0-18...|      1

In [ ]:
dataset_business_df = dataset_business_df.filter(dataset_business_df.is_open == 1)
dataset_business_df.show(truncate=False) 

+---------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------------------------------------------------------------+-------+-------------+---------------+---------------------------------+-----------+-----

In [ ]:
dataset_business_df.printSchema()

In [ ]:
business_df = dataset_business_df.select('business_id','name','address','categories','stars','city','latitude','longitude','postal_code','state')
business_df.show(truncate=False)

+----------------------+---------------------------------+---------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------+-------------+---------------+-----------+-----+
|business_id           |name                             |address                    |categories                                                                                                                                                                                                                                                  |stars|city          |latitude     |longitude      |postal_code|state|
+----------------------+---------------------------------+---------------------------+------------------------------------------------------------------------------------------------

In [ ]:
state_list = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI","ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI","MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC","ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD","TN","TX", "UT","VT", "VA","WA", "WV", "WI", "WY"]

business_df = business_df.filter(business_df.state.isin(state_list))
business_df.show()

+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|         business_id|                name|             address|          categories|stars|          city|     latitude|      longitude|postal_code|state|
+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|6iYb2HFDywm3zjuRg...| Oskar Blues Taproom|        921 Pearl St|Gastropubs, Food,...|  4.0|       Boulder|   40.0175444|   -105.2833481|      80302|   CO|
|tCbdrRPZA0oiIYSmH...|Flying Elephants ...| 7000 NE Airport Way|Salad, Soup, Sand...|  4.0|      Portland|45.5889058992|-122.5933307507|      97218|   OR|
|bvN78flM8NLprQ1a1...|      The Reclaimory|  4720 Hawthorne Ave|Antiques, Fashion...|  4.5|      Portland|45.5119069956|-122.6136928797|      97214|   OR|
|oaepsyvc0J17qwi8c...|         Great Clips|  2566 Enterprise Rd|Beauty

In [ ]:
business_df = business_df.withColumnRenamed('name','business_name')
business_df.show()

+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|         business_id|       business_name|             address|          categories|stars|          city|     latitude|      longitude|postal_code|state|
+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|6iYb2HFDywm3zjuRg...| Oskar Blues Taproom|        921 Pearl St|Gastropubs, Food,...|  4.0|       Boulder|   40.0175444|   -105.2833481|      80302|   CO|
|tCbdrRPZA0oiIYSmH...|Flying Elephants ...| 7000 NE Airport Way|Salad, Soup, Sand...|  4.0|      Portland|45.5889058992|-122.5933307507|      97218|   OR|
|bvN78flM8NLprQ1a1...|      The Reclaimory|  4720 Hawthorne Ave|Antiques, Fashion...|  4.5|      Portland|45.5119069956|-122.6136928797|      97214|   OR|
|oaepsyvc0J17qwi8c...|         Great Clips|  2566 Enterprise Rd|Beauty

In [ ]:
business_df.dtypes

[('business_id', 'string'),
 ('business_name', 'string'),
 ('address', 'string'),
 ('categories', 'string'),
 ('stars', 'double'),
 ('city', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('postal_code', 'string'),
 ('state', 'string')]

In [ ]:
# Drop null values
business_nonull_df = business_df.dropna()
business_nonull_df.show()

+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|         business_id|       business_name|             address|          categories|stars|          city|     latitude|      longitude|postal_code|state|
+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|6iYb2HFDywm3zjuRg...| Oskar Blues Taproom|        921 Pearl St|Gastropubs, Food,...|  4.0|       Boulder|   40.0175444|   -105.2833481|      80302|   CO|
|tCbdrRPZA0oiIYSmH...|Flying Elephants ...| 7000 NE Airport Way|Salad, Soup, Sand...|  4.0|      Portland|45.5889058992|-122.5933307507|      97218|   OR|
|bvN78flM8NLprQ1a1...|      The Reclaimory|  4720 Hawthorne Ave|Antiques, Fashion...|  4.5|      Portland|45.5119069956|-122.6136928797|      97214|   OR|
|oaepsyvc0J17qwi8c...|         Great Clips|  2566 Enterprise Rd|Beauty

In [ ]:
from pyspark.sql.types import IntegerType
business_nonull_df = business_nonull_df.withColumn("postal_code", business_nonull_df["postal_code"].cast(IntegerType()))
business_nonull_df.show()

+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|         business_id|       business_name|             address|          categories|stars|          city|     latitude|      longitude|postal_code|state|
+--------------------+--------------------+--------------------+--------------------+-----+--------------+-------------+---------------+-----------+-----+
|6iYb2HFDywm3zjuRg...| Oskar Blues Taproom|        921 Pearl St|Gastropubs, Food,...|  4.0|       Boulder|   40.0175444|   -105.2833481|      80302|   CO|
|tCbdrRPZA0oiIYSmH...|Flying Elephants ...| 7000 NE Airport Way|Salad, Soup, Sand...|  4.0|      Portland|45.5889058992|-122.5933307507|      97218|   OR|
|bvN78flM8NLprQ1a1...|      The Reclaimory|  4720 Hawthorne Ave|Antiques, Fashion...|  4.5|      Portland|45.5119069956|-122.6136928797|      97214|   OR|
|oaepsyvc0J17qwi8c...|         Great Clips|  2566 Enterprise Rd|Beauty

In [ ]:
business_nonull_df

In [11]:
business_nonull_df.dtypes

NameError: ignored

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://db-inst-on-aws.cdkwx2gwtmqa.us-east-2.rds.amazonaws.com:5432/db_on_aws"
config = {"user":"root",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

business_nonull_df.write.jdbc(url=jdbc_url, table='businessTable', mode=mode, properties=config)

In [ ]:
# Read in data from S3 Buckets
#url1 ="https://yelp-sentiment.s3.us-east-2.amazonaws.com/yelp_academic_dataset_review.json"
spark.sparkContext.addFile(url1)
dataset_review_df = spark.read.json(SparkFiles.get("yelp_academic_dataset_review.json"))

# Show DataFrame
dataset_review_df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|buF9druCkbuXLX526...|   1|2014-10-11 03:34:02|    1|lWC-xP3rd6obsecCY...|  4.0|Apparently Prides...|     3|ak0TdVmGKo4pwqdJS...|
|RA4V8pr014UyUbDvI...|   0|2015-07-03 20:38:25|    0|8bFej1QE5LXp4O05q...|  4.0|This store is pre...|     1|YoVfDbnISlW0f7abN...|
|_sS2LBIGNT5NQb6PD...|   0|2013-05-28 20:38:06|    0|NDhkzczKjLshODbqD...|  5.0|I called WVM on t...|     0|eC5evKn1TWDyHCyQA...|
|0AzLzHfOJgL7ROwhd...|   1|2010-01-08 02:29:15|    1|T5fAqjjFooT4V0OeZ...|  2.0|I've stayed at ma...|     1|SFQ1jcnGguO0LYWnb...|
|8zehGz9jnxPqXtOc7...|   0|2011-07-28 18:05:01|    0|sjm_uUcQVxab_EeLC...|  4.0|The food i

In [ ]:
dataset_review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [ ]:
review_df = dataset_review_df.select('business_id','date','text','stars')
review_df.show()

+--------------------+-------------------+--------------------+-----+
|         business_id|               date|                text|stars|
+--------------------+-------------------+--------------------+-----+
|buF9druCkbuXLX526...|2014-10-11 03:34:02|Apparently Prides...|  4.0|
|RA4V8pr014UyUbDvI...|2015-07-03 20:38:25|This store is pre...|  4.0|
|_sS2LBIGNT5NQb6PD...|2013-05-28 20:38:06|I called WVM on t...|  5.0|
|0AzLzHfOJgL7ROwhd...|2010-01-08 02:29:15|I've stayed at ma...|  2.0|
|8zehGz9jnxPqXtOc7...|2011-07-28 18:05:01|The food is alway...|  4.0|
|xGXzsc-hzam-VArK6...|2018-01-21 04:41:03|This place used t...|  1.0|
|EXOsmAB1s71WePlQk...|2006-04-16 02:58:44|The setting is pe...|  2.0|
|DbXHNl890xSXNiyRc...|2017-12-02 18:16:13|Probably one of t...|  5.0|
|mD-A9KOWADXvfrZfw...|2012-05-28 15:00:47|I am definitely a...|  4.0|
|EEHhKSxUvJkoPSzeG...|2014-05-07 18:10:21|I work in the Pru...|  5.0|
|WQFn1A7-UAA4JT5YW...|2017-09-08 23:26:10|They NEVER seem t...|  1.0|
|iZxfWlfNPXV_0S1xO..

In [ ]:
from pyspark.sql.functions import *
review_df= review_df.withColumn("review_date",to_date(col('date')).alias('ts').cast("date"))

review_df.show()

+--------------------+-------------------+--------------------+-----+-----------+
|         business_id|               date|                text|stars|review_date|
+--------------------+-------------------+--------------------+-----+-----------+
|buF9druCkbuXLX526...|2014-10-11 03:34:02|Apparently Prides...|  4.0| 2014-10-11|
|RA4V8pr014UyUbDvI...|2015-07-03 20:38:25|This store is pre...|  4.0| 2015-07-03|
|_sS2LBIGNT5NQb6PD...|2013-05-28 20:38:06|I called WVM on t...|  5.0| 2013-05-28|
|0AzLzHfOJgL7ROwhd...|2010-01-08 02:29:15|I've stayed at ma...|  2.0| 2010-01-08|
|8zehGz9jnxPqXtOc7...|2011-07-28 18:05:01|The food is alway...|  4.0| 2011-07-28|
|xGXzsc-hzam-VArK6...|2018-01-21 04:41:03|This place used t...|  1.0| 2018-01-21|
|EXOsmAB1s71WePlQk...|2006-04-16 02:58:44|The setting is pe...|  2.0| 2006-04-16|
|DbXHNl890xSXNiyRc...|2017-12-02 18:16:13|Probably one of t...|  5.0| 2017-12-02|
|mD-A9KOWADXvfrZfw...|2012-05-28 15:00:47|I am definitely a...|  4.0| 2012-05-28|
|EEHhKSxUvJkoPSz

In [ ]:
review_df = review_df.withColumnRenamed('stars','review_stars').withColumnRenamed('business_id','r_business_id').withColumnRenamed('text','review_text')
review_df.show()

+--------------------+-------------------+--------------------+------------+-----------+
|       r_business_id|               date|         review_text|review_stars|review_date|
+--------------------+-------------------+--------------------+------------+-----------+
|buF9druCkbuXLX526...|2014-10-11 03:34:02|Apparently Prides...|         4.0| 2014-10-11|
|RA4V8pr014UyUbDvI...|2015-07-03 20:38:25|This store is pre...|         4.0| 2015-07-03|
|_sS2LBIGNT5NQb6PD...|2013-05-28 20:38:06|I called WVM on t...|         5.0| 2013-05-28|
|0AzLzHfOJgL7ROwhd...|2010-01-08 02:29:15|I've stayed at ma...|         2.0| 2010-01-08|
|8zehGz9jnxPqXtOc7...|2011-07-28 18:05:01|The food is alway...|         4.0| 2011-07-28|
|xGXzsc-hzam-VArK6...|2018-01-21 04:41:03|This place used t...|         1.0| 2018-01-21|
|EXOsmAB1s71WePlQk...|2006-04-16 02:58:44|The setting is pe...|         2.0| 2006-04-16|
|DbXHNl890xSXNiyRc...|2017-12-02 18:16:13|Probably one of t...|         5.0| 2017-12-02|
|mD-A9KOWADXvfrZfw...

In [ ]:
review_df.dtypes

[('r_business_id', 'string'),
 ('date', 'string'),
 ('review_text', 'string'),
 ('review_stars', 'double'),
 ('review_date', 'date')]

In [ ]:
review_df = review_df.drop('date')
review_df.show()

+--------------------+--------------------+------------+-----------+
|       r_business_id|         review_text|review_stars|review_date|
+--------------------+--------------------+------------+-----------+
|buF9druCkbuXLX526...|Apparently Prides...|         4.0| 2014-10-11|
|RA4V8pr014UyUbDvI...|This store is pre...|         4.0| 2015-07-03|
|_sS2LBIGNT5NQb6PD...|I called WVM on t...|         5.0| 2013-05-28|
|0AzLzHfOJgL7ROwhd...|I've stayed at ma...|         2.0| 2010-01-08|
|8zehGz9jnxPqXtOc7...|The food is alway...|         4.0| 2011-07-28|
|xGXzsc-hzam-VArK6...|This place used t...|         1.0| 2018-01-21|
|EXOsmAB1s71WePlQk...|The setting is pe...|         2.0| 2006-04-16|
|DbXHNl890xSXNiyRc...|Probably one of t...|         5.0| 2017-12-02|
|mD-A9KOWADXvfrZfw...|I am definitely a...|         4.0| 2012-05-28|
|EEHhKSxUvJkoPSzeG...|I work in the Pru...|         5.0| 2014-05-07|
|WQFn1A7-UAA4JT5YW...|They NEVER seem t...|         1.0| 2017-09-08|
|iZxfWlfNPXV_0S1xO...|Disappointin

In [ ]:
from langdetect import detect
#import detect function from langdetect
nonen=review_df[review_df['review_text'].apply(detect)!='en']

ModuleNotFoundError: ignored

In [ ]:
# Write DataFrame to table

#review_df.write.jdbc(url=jdbc_url, table='reviewTable', mode=mode, properties=config)

In [ ]:
business_review_df = business_nonull_df.join(review_df,business_nonull_df.business_id ==  review_df.r_business_id,"inner")
business_review_df.show()

+--------------------+-------------+-----------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+--------------------+------------+-----------+
|         business_id|business_name|          address|          categories|stars|     city|  latitude|   longitude|postal_code|state|       r_business_id|         review_text|review_stars|review_date|
+--------------------+-------------+-----------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+--------------------+------------+-----------+
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|-36nnCT71XE0InJXK...|Customer service ...|         1.0| 2016-03-08|
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|-36nnCT71XE0InJXK...|Love my neighborh...|         5.0| 2017-04

In [ ]:
#Dropped 2 columns
business_review_df = business_review_df.drop('r_business_id')
business_review_df.show()

+--------------------+-------------+-----------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+------------+-----------+
|         business_id|business_name|          address|          categories|stars|     city|  latitude|   longitude|postal_code|state|         review_text|review_stars|review_date|
+--------------------+-------------+-----------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+------------+-----------+
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|Customer service ...|         1.0| 2016-03-08|
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|Love my neighborh...|         5.0| 2017-04-11|
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.641231

In [ ]:
#Removes ascii characters
from pyspark.sql.functions import udf

def ascii_ignore(x):
    if x:
      return x.encode('ascii', 'ignore').decode('ascii')
    else:
      return None 

ascii_udf = udf(ascii_ignore)

business_review_df.withColumn("reviews", ascii_udf('review_text')).show()

+--------------------+-------------+-----------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+------------+-----------+--------------------+
|         business_id|business_name|          address|          categories|stars|     city|  latitude|   longitude|postal_code|state|         review_text|review_stars|review_date|             reviews|
+--------------------+-------------+-----------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+------------+-----------+--------------------+
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|Customer service ...|         1.0| 2016-03-08|Customer service ...|
|-36nnCT71XE0InJXK...|      Safeway|2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|Love my neighborh...|         5.0| 2017-04-11|Love my neighborh

In [ ]:
#remove non-english words
import pyspark.sql.functions as F
import nltk
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

nltk.download('words')
nltk.download('wordnet')

@F.udf('array<string>')
def remove_words(words):
    return [word for word in words if wnl.lemmatize(word) in nltk.corpus.words.words()]

business_review_df = business_review_df.withColumn('review_text', remove_words('review_text'))
business_review_df.show()

In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 11.4MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=5024ae954acef736eeb0bb89844a96069b6b80be6f0e2b2879c84103a46408c4
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect


In [ ]:
from langdetect import detect
#import detect function from langdetect
#nonen=business_df.select(business_df['stars'].apply(detect))
#nonen.show()

business_review_df['lang'] = business_review_df['review_text'].apply(detect)
business_review_df.show()

In [3]:
import re
string = 'Hello!!! 45656~~~zhèng què. Yes. 正确｡'
new_string=re.sub('[^a-zA-Z0-9]',' ',string)

cleaned_string=re.sub('\s+',' ',new_string)

cleaned_string

'Hello 45656 zh ng qu Yes '

In [6]:
!pip install pyenchant

     |████████████████████████████████| 61kB 3.2MB/s 


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re
import string

def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 1 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 1:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 1:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    # remove unwanted space, *.split() will automatically split on
    # whitespace and discard duplicates, the " ".join() joins the
    # resulting list into one string.
    return " ".join(cleaned_str.split())
# setup pyspark udf function
remove_features_udf = udf(remove_features, StringType())

business_review_df = business_review_df.withColumn('removed',remove_features_udf(business_review_df['review_text']))
business_review_df.show(5,True)

KeyboardInterrupt: ignored

In [ ]:
#business_review_df.write.jdbc(url=jdbc_url, table='businessreviewtable', mode=mode, properties=config)

In [ ]:
business_review_df.filter(business_review_df.city == 'Vancouver').show(50)

+--------------------+-------------+--------------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+------------+-----------+
|         business_id|business_name|             address|          categories|stars|     city|  latitude|   longitude|postal_code|state|         review_text|review_stars|review_date|
+--------------------+-------------+--------------------+--------------------+-----+---------+----------+------------+-----------+-----+--------------------+------------+-----------+
|-36nnCT71XE0InJXK...|      Safeway|   2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|Customer service ...|         1.0| 2016-03-08|
|-36nnCT71XE0InJXK...|      Safeway|   2615 NE 112th Ave|Shopping, Drugsto...|  2.0|Vancouver|45.6412317|-122.5571137|      98684|   WA|Love my neighborh...|         5.0| 2017-04-11|
|-36nnCT71XE0InJXK...|      Safeway|   2615 NE 112th Ave|Shopping, Drugsto...|  2.0|V

In [ ]:
# Count after filtering by US State -- 8012598
business_filtered_by_state_df.count()

8012598

In [ ]:
#Count before filtering by US state excluding closed stores -- 8635403
business_review_df.count()

6471345

In [ ]:
#Count before filtering by US state -- 8635403
business_review_df.count()

8635403

In [ ]:
#business_filtered_by_state_df.write.option("header","true").csv("//yelp-sentiment.s3.us-east-2.amazonaws.com/business_review")

business_filtered_by_state_df.write.format('csv').option('header','true').save('s3a://yelp-sentiment.s3.us-east-2.amazonaws.com/business_review.csv',mode='overwrite')

Py4JJavaError: ignored

**************Project cleanup ends here****************

### Feature Transformations


In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [ ]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[33933,69...|
|  1.0|(262145,[15889,13...|
|  1.0|(262145,[25570,63...|
|  0.0|(262145,[6286,272...|
|  0.0|(262145,[6979,255...|
|  1.0|(262145,[24417,24...|
|  1.0|(262145,[12084,48...|
|  1.0|(262145,[3645,963...|
|  0.0|(262145,[53777,10...|
|  0.0|(262145,[138356,2...|
|  0.0|(262145,[24113,25...|
|  1.0|(262145,[68867,13...|
|  1.0|(262145,[24417,36...|
|  0.0|(262145,[18098,24...|
|  1.0|(262145,[24417,25...|
|  1.0|(262145,[24417,25...|
|  0.0|(262145,[31704,21...|
|  1.0|(262145,[25570,27...|
|  1.0|(262145,[12329,15...|
|  1.0|(262145,[8287,139...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|negative|"The servers went...|    97|  1.0|["the, servers, w...|["the, servers, w...|(262144,[50940,67...|(262144,[50940,67...|(262145,[50940,67...|[-1099.6356550268...|[1.79816566668390...|       1.0|
|negative| "not even a ""hello|    19|  1.0|["not, even, a, "...|["not, even, ""he...|(262144,[174966,2...|(262144,[174966,2...|(262145,[174966,2...|[-260.00407273415...|[0.11233471792373.

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.746936
